In [2]:
import requests
import lxml
import re
from selenium import webdriver

from bs4 import BeautifulSoup
from urllib.parse import parse_qs, urlparse

from dateutil.relativedelta import relativedelta
from datetime import datetime
import time

import numpy as np
import pandas as pd

In [3]:
# 네이버 tv, 영화, 방송, 공연 등의 카테고리를 dict.key로 추가. dict.value는 당연히 해당 탭의 [컨텐츠 제목]
# 만약 존재하지 않을 시 없는 채로 dict 생성.
# 나중에 10명의 인물들의 dict를 DataFrame으로 취합.

## 셀레늄 드라이버 시작

In [4]:
driver = webdriver.Chrome('D:/Google Drive/chromedriver_85.exe')
driver.implicitly_wait(1)

## 샘플 설정

In [12]:
#x = input('검색어:')
sample = ['장성규','김우빈','김연아','이순재','손흥민','안재홍','유재석','정준하','정형돈','장도연']
#name = sample[0]
#name

## 파싱 준비

In [6]:
def ready_parcing(name):
    driver.get('https://people.search.naver.com/')
    driver.find_element_by_name('query').send_keys(name)
    driver.find_element_by_xpath('//*[@id="search_form"]/fieldset/input').click()

    driver.find_element_by_xpath('//*[@id="content"]/div/div[2]/div[2]/div[1]/div[1]/div/div/a/strong').click()
    driver.implicitly_wait(1)

    cur_page = driver.page_source
    global soup
    soup = BeautifulSoup(cur_page, 'html.parser')
    

## 인물 직업, 경력, 수상, 학력 사항

In [7]:
def get_profile(name):
    
    df_awards = np.nan # 데이터프레임 초기화
    df_career = np.nan # 데이터프레임 초기화
    
    df = pd.DataFrame()
    df_awards = pd.DataFrame(data = None)
    df_career = pd.DataFrame(data = None)
    dict_ = {}
    dict_['Name'] = name

    job = soup.find_all('dl', class_ = 'who')[0].find_all('dd', class_ = 'sub')[0].text.strip()
    dict_['Job'] = job

    for i in soup.find_all('div', class_ = 'record'):

        label = i.find_all('h3', class_ = 'blind')[0].text.strip()

        if label == '경력사항':
            career = i.find_all('p')
            career_date = i.find_all('dt')
            career_list = []
            career_date_list = []
            for o in career:
                career_list.append(o.text.strip())

            for u in career_date:
                career_date_list.append(u.text.strip())      

            career_str = ' | '.join(career_list)  
            dict_['Career_cnt'] = len(career_list)

            df_career['Date'] = career_date_list
            df_career['Career'] = career_list
            df_career['Name'] = name

        elif label == '학력사항':
            education = i.find_all('dd')
            education_list = []
            for o in education:
                education_list.append(o.text.strip())
            education_str = ' | '.join(education_list)  
            dict_['Education'] = education_str

        elif label == '수상내역':
            award = i.find_all('p')
            award_date = i.find_all('dt')
            award_list = []
            award_date_list = []

            for o in award:
                award_list.append(o.text.strip())

            for u in award_date:
                award_date_list.append(u.text.strip())

            dict_['Award_cnt'] = len(award_list)

            df_awards['Date'] = award_date_list
            df_awards['Award'] = award_list
            df_awards['Name'] = name


    # 없는 탭이 있을 경우 nan 값으로 대체
    if 'Career_cnt' not in dict_.keys():
        dict_['Career_cnt'] = np.nan
    #     df_career = np.nan    
    if 'Education' not in dict_.keys():
        dict_['Education'] = np.nan
    if 'Award_cnt' not in dict_.keys():
        dict_['Award_cnt'] = np.nan

#     print(dict_.keys())
    df = pd.DataFrame.from_dict([dict_])

    return {'basic': df, 'awards' : df_awards, 'career': df_career}



In [27]:
ready_parcing(name)

In [28]:
get_profile(name)

{'basic':   Name          Job  Career_cnt   Education  Award_cnt
 0  장성규  방송연예인, 아나운서           1  서울시립대학교 경제          2,
 'awards':    Date                                              Award Name
 0  2019  MBC 방송연예대상 예능부문 남자 신인상 (마이 리틀 텔레비전 V2, 전지적 참견 시점)  장성규
 1  2019                MBC 방송연예대상 라디오부문 신인상 (굿모닝FM 장성규입니다)  장성규,
 'career':         Date     Career Name
 0  ~ 2019.03  JTBC 아나운서  장성규}

In [29]:
get_profile(name)['career']

,Date,Career,Name
0,~ 2019.03,JTBC 아나운서,장성규


In [30]:
get_profile(name)['basic']

,Name,Job,Career_cnt,Education,Award_cnt
0,장성규,"방송연예인, 아나운서",1,서울시립대학교 경제,2


In [14]:
ready_parcing('김연아')

In [15]:
get_profile('김연아')['basic']

,Name,Job,Career_cnt,Education,Award_cnt
0,김연아,전 피겨스케이팅선수,23,고려대학교 대학원 체육학 | 고려대학교 체육교육학 학사 | 수리고등학교 | 도장중학...,79


In [17]:
for i in

['장성규', '김우빈', '김연아', '이순재', '손흥민', '안재홍', '유재석', '정준하', '정형돈', '장도연']

In [22]:
model_basic = pd.DataFrame()
model_career = pd.DataFrame()
model_awards = pd.DataFrame()

for name in sample:
    print(name, '스크랩 중')
    ready_parcing(name)
    model_basic = model_basic.append(get_profile(name)['basic'])
    model_career = model_career.append(get_profile(name)['career'])
    model_awards = model_awards.append(get_profile(name)['awards'])

장성규 스크랩 중
김우빈 스크랩 중
김연아 스크랩 중
이순재 스크랩 중
손흥민 스크랩 중
안재홍 스크랩 중
유재석 스크랩 중
정준하 스크랩 중
정형돈 스크랩 중
장도연 스크랩 중


In [23]:
model_basic

,Name,Job,Career_cnt,Education,Award_cnt
0,장성규,"방송연예인, 아나운서",1.0,서울시립대학교 경제,2
0,김우빈,영화배우,12.0,NaN,8
0,김연아,전 피겨스케이팅선수,23.0,고려대학교 대학원 체육학 | 고려대학교 체육교육학 학사 | 수리고등학교 | 도장중학...,79
0,이순재,"탤런트, 영화배우",22.0,서울대학교 철학 학사 | 서울고등학교,19
0,손흥민,축구선수,11.0,NaN,19
0,안재홍,영화배우,NaN,NaN,4
0,유재석,"MC, 개그맨",NaN,서울예술대학교 방송연예학 중퇴 | 용문고등학교 | 수유중학교 | 유현초등학교,44
0,정준하,"MC, 영화배우",3.0,강서고등학교 | 대림중학교 | 대림초등학교,9
0,정형돈,"MC, 개그맨",1.0,인덕대학 방송연예학 | 부산전자공업고등학교,17
0,장도연,개그맨,NaN,경희대학교 시각디자인과 휴학,4


In [24]:
model_career

,Date,Career,Name
0,~ 2019.03,JTBC 아나운서,장성규
0,2018,평창동계올림픽 홍보대사,김우빈
1,2013.09,전주대학교 명예홍보대사,김우빈
2,2013.08,수원 정보과학축제 홍보대사,김우빈
3,연도없음,서울컬렉션 모델,김우빈
...,...,...,...
10,2009,U-17 청소년월드컵 국가대표,손흥민
0,2014.07,다일공동체 홍보대사,정준하
1,2007.04,범국민 지식재산권보호연합회 홍보대사,정준하
2,2004,소방방재본부 안전 홍보대사,정준하


In [25]:
model_awards

,Date,Award,Name
0,2019,"MBC 방송연예대상 예능부문 남자 신인상 (마이 리틀 텔레비전 V2, 전지적 참견 시점)",장성규
1,2019,MBC 방송연예대상 라디오부문 신인상 (굿모닝FM 장성규입니다),장성규
0,2014,제35회 청룡영화상 인기스타상,김우빈
1,연도없음,제51회 대종상영화제 하나금융스타상,김우빈
2,2014,제18회 부천국제판타스틱영화제 판타지아 어워드,김우빈
...,...,...,...
16,2002,KBS 개그맨선발대회 동상,정형돈
0,2019,"MBC 방송연예대상 베스트 엔터테이너상 (같이 펀딩, 호구의 연애)",장도연
1,2013,제6회 메이크업 페스티벌 어워드 문화예술인희극부문,장도연
2,2012,제20회 대한민국문화연예대상 개그 최우수상,장도연


In [28]:
model_basic.to_csv("model_basic.csv", index=False, encoding = 'utf-8-sig')
model_career.to_csv("model_career.csv", index=False, encoding = 'utf-8-sig')
model_awards.to_csv("model_awards.csv", index=False, encoding = 'utf-8-sig')

## 출연 내역 상세

In [210]:
# driver.find_element_by_xpath('//*[@id="listUI_76"]/li[1]/span[1]/a').text.strip()

In [11]:
#content > div > div.workact_wrap > div > div:nth-child(1) > div > div._panel_album > ul > li:nth-child(2) > span.tit_wrap._ellipsis > a > span.tit_main._text

1. workact_header를 찾는다.
2. 1.에서 '영화' 카테고리가 몇번째에 오는지 찾는다.
3. 상세 내역이 2. 순서를 따르므로 순서대로 다음을 가져온다.
    3.1. 제목
    3.2. 연도
    3.3. 역할 (없을 수도 있음)
4. 한페이지에 5개씩 보여주므로 다음 페이지 버튼 클릭 수는 workact_header 의 건수//5 
    4.1 정해진 횟수를 클릭하는게 아니라 try/except를 써서 다음 페이지 클릭 버튼이 없으면 continue/break/pass 를 써도 될 듯.
    

In [21]:
album_list = []
onair_list = []
filmo_list = []
book_list = []
etc_list = []

In [22]:
class Switch:
    def switch(self, arg):
        self.case_name = 'case_' + str(arg)
        self.case = getattr(self, self.case_name, lambda: 'default')
        return self.case()
    
    def case_방송(self):
        print('-----방송-----')
        
    def case_영화(self):
        print('-----영화-----')
        
    def case_음악(self):
        print('------음악-----')
        
    def case_도서(self):
        print('-----도서-----')
        
    def case_공연(self):
        print('-----공연-----')
        
    def case_기타(self):
        print('-----default-----')

In [33]:
for i in soup.find_all('dl', class_ = 'workact_header')[0].find_all('span', class_ = 'kywrd'):
#     print(i.text.strip())
    cate = re.compile('[가-힣]+').findall(str(i.text.strip()))[0]
    m = re.compile('[0-9]+').findall(str(i.text.strip()))[0]
    Switch().switch(cate)
    print(cate + m + '건')


-----방송-----
방송47건
-----영화-----
영화1건
-----공연-----
공연1건
-----도서-----
도서1건
-----default-----
기타3건


In [67]:
work_desc = soup.find_all('div', class_ = 'workact_dsc')[0]

In [94]:
# 시도 4
p = re.compile('workact .+')
workact = work_desc.find_all('div', class_ = p)

for i in workact:
    cate = i.find_all('strong')[0].text.strip()
    
    i.find_all()
    if cate == '방송':
        print('---', cate, '---')
        year = i.find_all('span', class_ = 'status_sub')[-1].text.strip()
        print(year)
        title = i.find_all('a', class_ ='tit')[0].text.strip()
        print(title)
        as_ = '/'.join(re.compile('[가-힣]+').findall(i.find_all('span', class_ = 'status_sub')[0].text.strip()))
        print(as_)

    elif cate == '영화':
        print('---', cate, '---')
        year = i.find_all('span', class_ = 'status_sub')[-1].text.strip()
        print(year)
        title = i.find_all('a', class_ ='tit')[0].text.strip()
        print(title)
        as_ = '/'.join(re.compile('[가-힣]+').findall(i.find_all('span', class_ = 'status_sub')[0].text.strip()))
        print(as_)
#         for i in range(1, int(soup.find_all('div', class_ = 'workact type5')[0].find_all('span', class_ = 'count')[0].text.strip()[-1])+1):
#         cur_page = driver.page_source
#         soup = BeautifulSoup(cur_page, 'html.parser')
        
#         try:
#             driver.find_element_by_xpath('//*[@id="pagination_76"]/span[4]/a').click()
#             print('[페이지 읽는 중...]')
#             time.sleep(1)
#         except:
#             print('[마지막 페이지 읽는 중...]')
#             time.sleep(1)
            
#         for o in soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'tit_wrap'):
#             try:
#                 print(o.text.strip())
#                 onair_list.append(o.text.strip())
#                  #if int(soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'count')[0].text.strip()[-1]) > 1:
#             except:
#                 print('끝: page out of range')
        
#     elif cate == '영화':
        

--- 방송 ---
2020
추석특집 - 라면 당기는 시간

--- 영화 ---
2017년
반드시 잡는다
우정출연/뉴스앵커


In [66]:
work_desc[3].text.strip()

'도서1건 내 인생이다 임마(오늘을 버텨내는 우리들에게)넥서스BOOKS2019.05.02. 이전이전{=pageNo} / {=totalPageNo}다음다음 도서 상세보기'

In [57]:
# 시도 3


TypeError: 'str' object cannot be interpreted as an integer

In [47]:
# 시도 1
category_list = work_desc.find_all('strong', class_ = 'workact_tit')
content_list = work_desc.find_all('ul')

for x in work_desc.find_all('h3'):
    label = x.find('strong').text.strip() # 방송
#     if 'workact_tit' in str(x):
#         label = x.find('strong',class_ = 'workact_tit').text.strip()
# #         print(i.find_all('strong', class_ = 'workact_tit')[1].text.strip())
#     else: continue
    print(label)
    if label == '방송':
        print('----'+label+'----')
        
        for i in work_desc.find_all('div'):
            i.get_text()
            
#             i.find_all('span', class_ = 'count')[0].text.strip()[-1])+1):
#                 cur_page = driver.page_source
#                 soup = BeautifulSoup(cur_page, 'html.parser')

#                 try:
#                     driver.find_element_by_xpath('//*[@id="pagination_76"]/span[4]/a').click()
#                     print('[페이지 읽는 중...]')
#                     time.sleep(1)
#                 except:
#                     print('[마지막 페이지 읽는 중...]')
#                     time.sleep(1)

#                 for o in soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'tit_wrap'):
#                     try:
#                         print(o.text.strip())
#                         onair_list.append(o.text.strip())
#                          #if int(soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'count')[0].text.strip()[-1]) > 1:
#                     except:
#                         print('끝: page out of range')
                        
    elif label == '영화':
        print('----'+label+'----')
        
    
    elif label == '음악':
        print('----'+label+'----')
    
    elif label == '도서':
        print('----'+label+'----')
        
    elif label == '공연':
        print('----'+label+'----')
    
    elif label == '기타':
        print('----'+label+'----')
    
    
    
    
    else:
        print('뭔가 잘못됐다.')
        
        
        
        
        
        
        

방송


In [17]:
# 시도 2
if label in category_list:


for i, o in zip(category_list, content_list):
    print(i.text.strip())
    for z in o.find_all('li'):
        print(z.text.strip())
        #print(z.find_all('span')[0])

IndentationError: expected an indented block (<ipython-input-17-66e1863ba1c8>, line 4)

In [41]:
# 시도 0
# 방송 리스트
label = soup.find('div', class_ = 'workact type5').find_all('strong', class_ = 'workact_tit')[0].text.strip()
if label == '방송':
    for i in range(1, int(soup.find_all('div', class_ = 'workact type5')[0].find_all('span', class_ = 'count')[0].text.strip()[-1])+1):
        cur_page = driver.page_source
        soup = BeautifulSoup(cur_page, 'html.parser')
        
        try:
            driver.find_element_by_xpath('//*[@id="pagination_76"]/span[4]/a').click()
            print('[페이지 읽는 중...]')
            time.sleep(1)
        except:
            print('[마지막 페이지 읽는 중...]')
            time.sleep(1)
            
        for o in soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'tit_wrap'):
            try:
                print(o.text.strip())
                onair_list.append(o.text.strip())
                 #if int(soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'count')[0].text.strip()[-1]) > 1:
            except:
                print('끝: page out of range')
        

In [39]:
# 영화 리스트
label = soup.find_all('div', class_ = 'workact type5')[1].find_all('strong', class_ = 'workact_tit')[0].text.strip()
if label == '영화':
    for i in range(1, int(soup.find_all('div', class_ = 'workact type5')[1].find_all('span', class_ = 'count')[0].text.strip()[-1])+1):
        cur_page = driver.page_source
        soup = BeautifulSoup(cur_page, 'html.parser')

        try:
            driver.find_element_by_xpath('//*[@id="pagination_76"]/span[4]/a').click()
            print('[페이지 읽는 중...]')
            time.sleep(1)
        except:
            print('[마지막 페이지 읽는 중...]')
            time.sleep(1)

        for o in soup.find_all('div', class_ = 'workact type5')[1].find_all('span', class_ = 'tit_wrap'):
            try:
                print(o.text.strip())
                filmo_list.append(o.text.strip())
                 #if int(soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'count')[0].text.strip()[-1]) > 1:
            except:
                print('끝: page out of range')

if label == '방송':
    
       for i in range(1, int(soup.find_all('div', class_ = 'workact type5')[1].find_all('span', class_ = 'count')[0].text.strip()[-1])+1):
        cur_page = driver.page_source
        soup = BeautifulSoup(cur_page, 'html.parser')
        
        try:
            driver.find_element_by_xpath('//*[@id="pagination_76"]/span[4]/a').click()
            print('[페이지 읽는 중...]')
            time.sleep(1)
        except:
            print('[마지막 페이지 읽는 중...]')
            time.sleep(1)

        for o in soup.find_all('div', class_ = 'workact type5')[1].find_all('span', class_ = 'tit_wrap'):
            try:
                print(o.text.strip())
                filmo_list.append(o.text.strip())
                 #if int(soup.find('div', class_ = 'workact type5').find_all('span', class_ = 'count')[0].text.strip()[-1]) > 1:
            except:
                print('끝: page out of range')


ValueError: invalid literal for int() with base 10: '}'

In [ ]:
driver.page_source